### Notebook Used for Capstone Project

## Description of the problem and a discussion of the background


#### Background

During the Budget 2021, the goal of achieving 60,000 EV charging points at public carparks (40,000) and private premises (20,000) by 2030 was announced, as well as other measures  to support the adoption of electric vehicles (EVs). These include lowering the Additional Registration Fee (ARF) floor for fully electric cars and adjusting road tax for electric cars.

By 2025, eight “EV-Ready Towns” across Singapore would be fitted with EV charging points (Ang Mo Kio, Bedok, Choa Chu Kang, Jurong West, Punggol, Queenstown, Sembawang and Tengah).These locations are well spread out across the island, and have a high concentration of car parks with existing electrical capacity to support the deployment of charging points.

#### Challenge

However, it is possible that these 40,000 EV charging points in HDB carparks might not be sufficient. Other than electrical capacity and grid contraints, we will be undertaking analysis on the number of existing carparks, number of hdb unit dwellings in the different estates as well as the number of employed residents by work place region, whose usual mode of transport to work is by car only. We will also be considering nearby places around carparks, where time could be spent at the office, dining, weekly errands or leisure activities since charging points will likely be slow chargers which require time to charge.

Hence, these potential EV charging points could also be areas of consideration.


## Description of the data and how it will be used to solve the problem

4 different data sets will be used to analyse this issue:
1. List of HDB Carparks ('hdb-carpark-information.csv',source: https://data.gov.sg)
2. List HDB Dwelling Units ('hdb-property-information.csv', source: https://data.gov.sg)
3. Number of Employed Residents who only use car to travel to work by Workplace Region ('t111-118 .xlsx', https://www.singstat.gov.sg/)
4. Demographic data by Planning Area (



https://www.lta.gov.sg/content/ltagov/en/industry_innovations/technologies/electric_vehicles.html/#ev_charger_deployment

In [8]:
#Importing relevant libraries
import pandas as pd
import numpy as np
import folium
from folium.plugins import MarkerCluster
import json
from pandas.io.json import json_normalize
import requests
from bs4 import BeautifulSoup
from pyproj import Proj, transform
#from pyproj import Transformer, transform
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm

from geopy.geocoders import Nominatim

import geopandas as gpd
#import branca.colormap as cm
import utm
import os

from sklearn.cluster import KMeans


### The 8 Towns where EV charging points will be built: Ang Mo Kio, Bedok, Choa Chu Kang, Jurong West, Punggol, Queenstown, Sembawang, Tengah 

In [12]:
ev_location=pd.read_csv('ev_location.csv')

In [13]:
ev_location

,planning_area_of_residence,area_longitude,area_latitude
0,CHOA CHU KANG,103.747191,1.385556
1,SEMBAWANG,103.818828,1.455747
2,BEDOK,103.928409,1.324047
3,ANG MO KIO,103.842097,1.380092
4,PUNGGOL,103.910043,1.405363
5,QUEENSTOWN,103.784559,1.289073
6,JURONG WEST,103.704903,1.341612


In [15]:
#creating separate dataframe for each EV town
bedok=ev_location[(ev_location['planning_area_of_residence']=='BEDOK')]
bedok.reset_index(inplace=True,drop=True)
amk=ev_location[(ev_location['planning_area_of_residence']=='ANG MO KIO')]
amk.reset_index(inplace=True,drop=True)
punggol=ev_location[(ev_location['planning_area_of_residence']=='PUNGGOL')]
punggol.reset_index(inplace=True,drop=True)
queenstown=ev_location[(ev_location['planning_area_of_residence']=='QUEENSTOWN')]
queenstown.reset_index(inplace=True,drop=True)
jw=ev_location[(ev_location['planning_area_of_residence']=='JURONG WEST')]
jw.reset_index(inplace=True,drop=True)
semb=ev_location[(ev_location['planning_area_of_residence']=='SEMBAWANG')]
semb.reset_index(inplace=True,drop=True)
cck=ev_location[(ev_location['planning_area_of_residence']=='CHOA CHU KANG')]
cck.reset_index(inplace=True,drop=True)
tengah=ev_location[(ev_location['planning_area_of_residence']=='TENGAH')]
tengah.reset_index(inplace=True,drop=True)


## Exploring Carpark Areas

To maximize the 100 limit output provided by the Foursquare free API each EV town will be analyzed separately and  the 8 dataframes concatenated.


## BEDOK

In [16]:
# @hidden_cell
#Defining Foursquare credentials and version
CLIENT_ID='5GYCGVHIUVEFMT10VNQK1EHYW4XAJIILTGVWUATISPOTWQ3K'
CLIENT_SECRET='UXNQUS20PDIZSMSXW1NMHZXC55XGYYG5IRUCYXHSAD4TXVSR'
VERSION='20180605'

In [18]:
#create GET request URL

latitude=bedok.loc[0,'area_latitude']
longitude=bedok.loc[0,'area_longitude']

LIMIT=200
radius=1000

url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID,
CLIENT_SECRET,
VERSION,
latitude,
longitude,
radius,
LIMIT)

url


'https://api.foursquare.com/v2/venues/explore?&client_id=5GYCGVHIUVEFMT10VNQK1EHYW4XAJIILTGVWUATISPOTWQ3K&client_secret=UXNQUS20PDIZSMSXW1NMHZXC55XGYYG5IRUCYXHSAD4TXVSR&v=20180605&ll=1.3240468743815126,103.92840945179222&radius=1000&limit=200'

In [19]:
#send GET request url and examine results
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '613da88984ec8e72cc691bff'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bedok',
  'headerFullLocation': 'Bedok, Singapore',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 107,
  'suggestedBounds': {'ne': {'lat': 1.3330468833815217,
    'lng': 103.9373950559067},
   'sw': {'lat': 1.3150468653815035, 'lng': 103.91942384767773}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4dca91d952b1c2222a85852c',
       'name': 'Bedok Chwee Kueh 勿洛水粿',
       'location': {'address': '#01-19 Bedok Interchange Hawker Centre',
        'crossStreet': '207 New Upper Changi Rd',
        'lat': 1.3249029,
        'lng': 103.93025,
        'lab

In [20]:
#function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']
    

In [21]:
#Cleaning the json and structuring it into a Pandas DataFrame
venues = results['response']['groups'][0]['items']

#flatten json
nearby_venues = pd.json_normalize(venues)
                       
#filter columns
filtered_columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues=nearby_venues.loc[:,filtered_columns]

#filter category for each row
nearby_venues['venue.categories']=nearby_venues.apply(get_category_type, axis=1)
                           
#clean columns
nearby_venues.columns=[col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(200)

,name,categories,lat,lng
0,Bedok Chwee Kueh 勿洛水粿,Chinese Restaurant,1.324903,103.930250
1,Haidilao Hot Pot 海底捞火锅,Hotpot Restaurant,1.324299,103.929104
2,Ya Kun Kaya Toast 亞坤,Coffee Shop,1.324095,103.929198
3,Song Zhou Luo Bo Gao 松洲箩卜糕,Breakfast Spot,1.324836,103.930520
4,FairPrice Fínest,Supermarket,1.324140,103.929260
...,...,...,...,...
95,Food Hub @ Chai Chee 福阁食厅,Food Court,1.323398,103.923092
96,Chai Chee Minced Meat Noodle 菜市肉脞面,Noodle House,1.326942,103.933042
97,Playground @ Jalan Bangsawan,Playground,1.321908,103.923544
98,Burgs by Project Warung,Burger Joint,1.323437,103.922390


In [22]:
#create 3 additional columns to indicate respective planning area and its longitude and latitude
bedok=nearby_venues.assign(planning_area='BEDOK',
                           area_latitude=latitude,
                           area_longitude=longitude)


In [23]:
#Repositioning the columns and renaming the column names

bedok=bedok[['planning_area','area_latitude','area_longitude','name','lat','lng','categories']]

bedok.rename(columns={'name':'venue_name',
            'lat':'venue_latitude',
             'lng':'venue_longitude',
             'categories':'venue_categories'}, inplace=True)

In [25]:
# Check data
bedok

,planning_area,area_latitude,area_longitude,venue_name,venue_latitude,venue_longitude,venue_categories
0,BEDOK,1.324047,103.928409,Bedok Chwee Kueh 勿洛水粿,1.324903,103.930250,Chinese Restaurant
1,BEDOK,1.324047,103.928409,Haidilao Hot Pot 海底捞火锅,1.324299,103.929104,Hotpot Restaurant
2,BEDOK,1.324047,103.928409,Ya Kun Kaya Toast 亞坤,1.324095,103.929198,Coffee Shop
3,BEDOK,1.324047,103.928409,Song Zhou Luo Bo Gao 松洲箩卜糕,1.324836,103.930520,Breakfast Spot
4,BEDOK,1.324047,103.928409,FairPrice Fínest,1.324140,103.929260,Supermarket
...,...,...,...,...,...,...,...
95,BEDOK,1.324047,103.928409,Food Hub @ Chai Chee 福阁食厅,1.323398,103.923092,Food Court
96,BEDOK,1.324047,103.928409,Chai Chee Minced Meat Noodle 菜市肉脞面,1.326942,103.933042,Noodle House
97,BEDOK,1.324047,103.928409,Playground @ Jalan Bangsawan,1.321908,103.923544,Playground
98,BEDOK,1.324047,103.928409,Burgs by Project Warung,1.323437,103.922390,Burger Joint


In [26]:
bedok.to_csv("bedok.csv")

## JURONG WEST

In [16]:
# @hidden_cell
#Defining Foursquare credentials and version
CLIENT_ID='5GYCGVHIUVEFMT10VNQK1EHYW4XAJIILTGVWUATISPOTWQ3K'
CLIENT_SECRET='UXNQUS20PDIZSMSXW1NMHZXC55XGYYG5IRUCYXHSAD4TXVSR'
VERSION='20180605'

In [27]:
# Create GET request url
latitude=jw.loc[0,'area_latitude']
longitude=jw.loc[0,'area_longitude']

LIMIT=200
radius=1000

url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID,
CLIENT_SECRET,
VERSION,
latitude,
longitude,
radius,
LIMIT)

url


'https://api.foursquare.com/v2/venues/explore?&client_id=5GYCGVHIUVEFMT10VNQK1EHYW4XAJIILTGVWUATISPOTWQ3K&client_secret=UXNQUS20PDIZSMSXW1NMHZXC55XGYYG5IRUCYXHSAD4TXVSR&v=20180605&ll=1.3416119936431667,103.70490266187328&radius=1000&limit=200'

In [28]:
#send GET request url and examine results
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '613da8b2d1665508d2cdfa2b'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Jurong West',
  'headerFullLocation': 'Jurong West, Singapore',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 78,
  'suggestedBounds': {'ne': {'lat': 1.3506120026431758,
    'lng': 103.71388833008032},
   'sw': {'lat': 1.3326119846431577, 'lng': 103.69591699366623}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bf92990b182c9b6bf7a785a',
       'name': "Carl's Jr.",
       'location': {'address': '#02-42, Jurong Point (JP1)',
        'crossStreet': '1 Jurong West Central 2',
        'lat': 1.340574772014918,
        'lng': 103.70638409348805,
      

In [29]:
#function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']
    

In [30]:
#Cleaning the json and structuring it into a Pandas DataFrame
venues = results['response']['groups'][0]['items']

#flatten json
nearby_venues = pd.json_normalize(venues)
                       
#filter columns
filtered_columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues=nearby_venues.loc[:,filtered_columns]

#filter category for each row
nearby_venues['venue.categories']=nearby_venues.apply(get_category_type, axis=1)
                           
#clean columns
nearby_venues.columns=[col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(200)

,name,categories,lat,lng
0,Carl's Jr.,Fast Food Restaurant,1.340575,103.706384
1,Subway,Sandwich Place,1.340278,103.706548
2,Ya Kun Kaya Toast,Breakfast Spot,1.339619,103.705786
3,PUTIEN Restaurant 莆田菜馆,Chinese Restaurant,1.340400,103.706468
4,Din Tai Fung 鼎泰豐 (Din Tai Fung),Chinese Restaurant,1.339029,103.705765
...,...,...,...,...
73,Playground@jurong West St 61,Playground,1.336403,103.698483
74,Juz Bread @ Pioneer Mall,Bakery,1.341754,103.696443
75,24hr Coffeeshop @ Jalan Bahar,Coffee Shop,1.350099,103.704962
76,Singapore Boys' Home,Hostel,1.349702,103.707644


In [31]:
#create 3 additional columns to indicate respective planning area and its longitude and latitude
jw=nearby_venues.assign(planning_area='JURONG WEST',
                           area_latitude=latitude,
                           area_longitude=longitude)


In [32]:
jw=jw[['planning_area','area_latitude','area_longitude','name','lat','lng','categories']]

In [33]:

jw.rename(columns={'name':'venue_name',
            'lat':'venue_latitude',
             'lng':'venue_longitude',
             'categories':'venue_categories'}, inplace=True)

In [34]:
jw

,planning_area,area_latitude,area_longitude,venue_name,venue_latitude,venue_longitude,venue_categories
0,JURONG WEST,1.341612,103.704903,Carl's Jr.,1.340575,103.706384,Fast Food Restaurant
1,JURONG WEST,1.341612,103.704903,Subway,1.340278,103.706548,Sandwich Place
2,JURONG WEST,1.341612,103.704903,Ya Kun Kaya Toast,1.339619,103.705786,Breakfast Spot
3,JURONG WEST,1.341612,103.704903,PUTIEN Restaurant 莆田菜馆,1.340400,103.706468,Chinese Restaurant
4,JURONG WEST,1.341612,103.704903,Din Tai Fung 鼎泰豐 (Din Tai Fung),1.339029,103.705765,Chinese Restaurant
...,...,...,...,...,...,...,...
73,JURONG WEST,1.341612,103.704903,Playground@jurong West St 61,1.336403,103.698483,Playground
74,JURONG WEST,1.341612,103.704903,Juz Bread @ Pioneer Mall,1.341754,103.696443,Bakery
75,JURONG WEST,1.341612,103.704903,24hr Coffeeshop @ Jalan Bahar,1.350099,103.704962,Coffee Shop
76,JURONG WEST,1.341612,103.704903,Singapore Boys' Home,1.349702,103.707644,Hostel


In [35]:
jw.to_csv("jw.csv")

## CHOA CHU KANG

In [36]:
# @hidden_cell
#Defining Foursquare credentials and version
CLIENT_ID='5GYCGVHIUVEFMT10VNQK1EHYW4XAJIILTGVWUATISPOTWQ3K'
CLIENT_SECRET='UXNQUS20PDIZSMSXW1NMHZXC55XGYYG5IRUCYXHSAD4TXVSR'
VERSION='20180605'

In [37]:
#Create GET request url

latitude=cck.loc[0,'area_latitude']
longitude=cck.loc[0,'area_longitude']

LIMIT=200
radius=1000

url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID,
CLIENT_SECRET,
VERSION,
latitude,
longitude,
radius,
LIMIT)

url


'https://api.foursquare.com/v2/venues/explore?&client_id=5GYCGVHIUVEFMT10VNQK1EHYW4XAJIILTGVWUATISPOTWQ3K&client_secret=UXNQUS20PDIZSMSXW1NMHZXC55XGYYG5IRUCYXHSAD4TXVSR&v=20180605&ll=1.385555907688099,103.74719055092544&radius=1000&limit=200'

In [38]:
#send GET request url and examine results

results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '613da8c8d6eeb73204c97ecc'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Choa Chu Kang',
  'headerFullLocation': 'Choa Chu Kang, Singapore',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 42,
  'suggestedBounds': {'ne': {'lat': 1.394555916688108,
    'lng': 103.75617638318077},
   'sw': {'lat': 1.37655589868809, 'lng': 103.7382047186701}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4f6332c6e4b08b476e2604b1',
       'name': 'パン達人 BAKER TALENT',
       'location': {'address': '303 Choa Chu Kang Ave 4 #01-723',
        'lat': 1.384180192719575,
        'lng': 103.74423274608141,
        'labeledLatLngs': [{'label': 'displa

In [39]:
#function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']
    

In [40]:
#Cleaning the json and structuring it into a Pandas DataFrame
venues = results['response']['groups'][0]['items']

#flatten json
nearby_venues = pd.json_normalize(venues)
                       
#filter columns
filtered_columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues=nearby_venues.loc[:,filtered_columns]

#filter category for each row
nearby_venues['venue.categories']=nearby_venues.apply(get_category_type, axis=1)
                           
#clean columns
nearby_venues.columns=[col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(200)

,name,categories,lat,lng
0,パン達人 BAKER TALENT,Bakery,1.384180,103.744233
1,Subway,Sandwich Place,1.385643,103.744725
2,Starbucks,Coffee Shop,1.384324,103.744666
3,Dian Xiao Er 店小二,Asian Restaurant,1.385604,103.744748
4,Siam Kitchen,Thai Restaurant,1.385240,103.744889
5,Choa Chu Kang Swimming Complex,Pool,1.391483,103.747921
6,Limbang Park,Park,1.389905,103.744499
7,Ya Kun Kaya Toast 亞坤,Coffee Shop,1.385537,103.744747
8,Choa Chu Kang Sports & Recreation Centre,Gym / Fitness Center,1.391601,103.747831
9,Roof Top Garden,Playground,1.385268,103.744831


In [41]:
#create 3 additional columns to indicate respective planning area and its longitude and latitude
cck=nearby_venues.assign(planning_area='CHOA CHU KANG',
                           area_latitude=latitude,
                           area_longitude=longitude)


In [42]:
#Repositioning the columns and renaming the column names
cck=cck[['planning_area','area_latitude','area_longitude','name','lat','lng','categories']]
cck.rename(columns={'name':'venue_name',
            'lat':'venue_latitude',
             'lng':'venue_longitude',
             'categories':'venue_categories'}, inplace=True)

In [44]:
#check data
cck

,planning_area,area_latitude,area_longitude,venue_name,venue_latitude,venue_longitude,venue_categories
0,CHOA CHU KANG,1.385556,103.747191,パン達人 BAKER TALENT,1.384180,103.744233,Bakery
1,CHOA CHU KANG,1.385556,103.747191,Subway,1.385643,103.744725,Sandwich Place
2,CHOA CHU KANG,1.385556,103.747191,Starbucks,1.384324,103.744666,Coffee Shop
3,CHOA CHU KANG,1.385556,103.747191,Dian Xiao Er 店小二,1.385604,103.744748,Asian Restaurant
4,CHOA CHU KANG,1.385556,103.747191,Siam Kitchen,1.385240,103.744889,Thai Restaurant
5,CHOA CHU KANG,1.385556,103.747191,Choa Chu Kang Swimming Complex,1.391483,103.747921,Pool
6,CHOA CHU KANG,1.385556,103.747191,Limbang Park,1.389905,103.744499,Park
7,CHOA CHU KANG,1.385556,103.747191,Ya Kun Kaya Toast 亞坤,1.385537,103.744747,Coffee Shop
8,CHOA CHU KANG,1.385556,103.747191,Choa Chu Kang Sports & Recreation Centre,1.391601,103.747831,Gym / Fitness Center
9,CHOA CHU KANG,1.385556,103.747191,Roof Top Garden,1.385268,103.744831,Playground


In [45]:
cck.to_csv("cck.csv")

## PUNGGOL

In [46]:
# @hidden_cell
#Defining Foursquare credentials and version
CLIENT_ID='5GYCGVHIUVEFMT10VNQK1EHYW4XAJIILTGVWUATISPOTWQ3K'
CLIENT_SECRET='UXNQUS20PDIZSMSXW1NMHZXC55XGYYG5IRUCYXHSAD4TXVSR'
VERSION='20180605'

In [47]:
# Create GET request url

latitude=punggol.loc[0,'area_latitude']
longitude=punggol.loc[0,'area_longitude']

LIMIT=200
radius=1000

url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID,
CLIENT_SECRET,
VERSION,
latitude,
longitude,
radius,
LIMIT)

url


'https://api.foursquare.com/v2/venues/explore?&client_id=5GYCGVHIUVEFMT10VNQK1EHYW4XAJIILTGVWUATISPOTWQ3K&client_secret=UXNQUS20PDIZSMSXW1NMHZXC55XGYYG5IRUCYXHSAD4TXVSR&v=20180605&ll=1.405362997897329,103.91004339487458&radius=1000&limit=200'

In [48]:
#send GET request url and examine results

results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '613da8e8cb2e670bcf59ed2f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Punggol',
  'headerFullLocation': 'Punggol, Singapore',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 68,
  'suggestedBounds': {'ne': {'lat': 1.414363006897338,
    'lng': 103.91902930280251},
   'sw': {'lat': 1.39636298889732, 'lng': 103.90105748694664}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4de2614c52b1d38d296c044a',
       'name': 'Punggol Waterway Park',
       'location': {'address': 'Punggol',
        'lat': 1.4102274642652444,
        'lng': 103.90697479248047,
        'labeledLatLngs': [{'label': 'display',
          'lat': 1.4102274

In [49]:
#function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']
    

In [50]:
#Cleaning the json and structuring it into a Pandas DataFrame
venues = results['response']['groups'][0]['items']

#flatten json
nearby_venues = pd.json_normalize(venues)
                       
#filter columns
filtered_columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues=nearby_venues.loc[:,filtered_columns]

#filter category for each row
nearby_venues['venue.categories']=nearby_venues.apply(get_category_type, axis=1)
                           
#clean columns
nearby_venues.columns=[col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(200)

,name,categories,lat,lng
0,Punggol Waterway Park,Park,1.410227,103.906975
1,Waterway Point,Shopping Mall,1.405425,103.902411
2,Shaw Theatres,Multiplex,1.406948,103.902423
3,Uniqlo,Clothing Store,1.406721,103.901837
4,FairPrice Finest,Supermarket,1.407055,103.902348
...,...,...,...,...
63,Toy Or Game,Toy / Game Store,1.406679,103.901867
64,7-Eleven,Convenience Store,1.399582,103.916005
65,Roof Garden @ Blk 106c,Park,1.397261,103.907262
66,Master Crab 螃蟹师傅,Coffee Shop,1.396733,103.908992


In [51]:
#create 3 additional columns to indicate respective planning area and its longitude and latitude
punggol=nearby_venues.assign(planning_area='PUNGGOL',
                           area_latitude=latitude,
                           area_longitude=longitude)


In [52]:
#Repositioning the columns and renaming the column names
punggol=punggol[['planning_area','area_latitude','area_longitude','name','lat','lng','categories']]

punggol.rename(columns={'name':'venue_name',
            'lat':'venue_latitude',
             'lng':'venue_longitude',
             'categories':'venue_categories'}, inplace=True)

In [54]:
#check data
punggol

,planning_area,area_latitude,area_longitude,venue_name,venue_latitude,venue_longitude,venue_categories
0,PUNGGOL,1.405363,103.910043,Punggol Waterway Park,1.410227,103.906975,Park
1,PUNGGOL,1.405363,103.910043,Waterway Point,1.405425,103.902411,Shopping Mall
2,PUNGGOL,1.405363,103.910043,Shaw Theatres,1.406948,103.902423,Multiplex
3,PUNGGOL,1.405363,103.910043,Uniqlo,1.406721,103.901837,Clothing Store
4,PUNGGOL,1.405363,103.910043,FairPrice Finest,1.407055,103.902348,Supermarket
...,...,...,...,...,...,...,...
63,PUNGGOL,1.405363,103.910043,Toy Or Game,1.406679,103.901867,Toy / Game Store
64,PUNGGOL,1.405363,103.910043,7-Eleven,1.399582,103.916005,Convenience Store
65,PUNGGOL,1.405363,103.910043,Roof Garden @ Blk 106c,1.397261,103.907262,Park
66,PUNGGOL,1.405363,103.910043,Master Crab 螃蟹师傅,1.396733,103.908992,Coffee Shop


In [55]:
punggol.to_csv("punggol.csv")

## ANG MO KIO

In [56]:
# @hidden_cell
#Defining Foursquare credentials and version
CLIENT_ID='5GYCGVHIUVEFMT10VNQK1EHYW4XAJIILTGVWUATISPOTWQ3K'
CLIENT_SECRET='UXNQUS20PDIZSMSXW1NMHZXC55XGYYG5IRUCYXHSAD4TXVSR'
VERSION='20180605'

In [57]:
#Create GET request url

latitude=amk.loc[0,'area_latitude']
longitude=amk.loc[0,'area_longitude']

LIMIT=200
radius=1000

url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID,
CLIENT_SECRET,
VERSION,
latitude,
longitude,
radius,
LIMIT)

url


'https://api.foursquare.com/v2/venues/explore?&client_id=5GYCGVHIUVEFMT10VNQK1EHYW4XAJIILTGVWUATISPOTWQ3K&client_secret=UXNQUS20PDIZSMSXW1NMHZXC55XGYYG5IRUCYXHSAD4TXVSR&v=20180605&ll=1.380092406326446,103.842096537105&radius=1000&limit=200'

In [58]:
#send GET request url and examine results

results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '613da906d1665508d2cea284'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Ang Mo Kio',
  'headerFullLocation': 'Ang Mo Kio, Singapore',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 39,
  'suggestedBounds': {'ne': {'lat': 1.389092415326455,
    'lng': 103.85108234867631},
   'sw': {'lat': 1.3710923973264368, 'lng': 103.83311072553367}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '57ced66f498eb312cf9a5f15',
       'name': 'Prodigal Roasters',
       'location': {'address': 'Blk 625, Ang Mo Kio Ave 9',
        'lat': 1.3822120650112213,
        'lng': 103.84134650230408,
        'labeledLatLngs': [{'label': 'display',
    

In [59]:
#function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']
    

In [60]:
#Cleaning the json and structuring it into a Pandas DataFrame
venues = results['response']['groups'][0]['items']

#flatten json
nearby_venues = pd.json_normalize(venues)
                       
#filter columns
filtered_columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues=nearby_venues.loc[:,filtered_columns]

#filter category for each row
nearby_venues['venue.categories']=nearby_venues.apply(get_category_type, axis=1)
                           
#clean columns
nearby_venues.columns=[col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(200)

,name,categories,lat,lng
0,Prodigal Roasters,Coffee Shop,1.382212,103.841347
1,Ho-Ree Roasted Food 好味烧腊,Chinese Restaurant,1.381160,103.840977
2,Vegetarian @ 630 AMK Kopitiam,Vegetarian / Vegan Restaurant,1.380721,103.841327
3,Ang Mo Kio Town Garden West,Park,1.376039,103.844528
4,Shanghai Ren Jia 上海人家,Shanghai Restaurant,1.377068,103.841329
5,Mr Teh Tarik Eating House,Food Court,1.378458,103.836685
6,Sumo Big Prawn,Noodle House,1.381121,103.840778
7,Charcos The Flaming Chicken,Australian Restaurant,1.378220,103.836871
8,Nanyang Polytechnic Auditorium,College Auditorium,1.380186,103.848298
9,Mr Teh Tarik,Halal Restaurant,1.372168,103.845602


In [61]:
#create 3 additional columns to indicate respective planning area and its longitude and latitude
amk=nearby_venues.assign(planning_area='ANG MO KIO',
                           area_latitude=latitude,
                           area_longitude=longitude)


In [62]:
#Repositioning the columns and renaming the column names

amk=amk[['planning_area','area_latitude','area_longitude','name','lat','lng','categories']]
amk.rename(columns={'name':'venue_name',
            'lat':'venue_latitude',
             'lng':'venue_longitude',
             'categories':'venue_categories'}, inplace=True)

In [63]:
#check data
amk

,planning_area,area_latitude,area_longitude,venue_name,venue_latitude,venue_longitude,venue_categories
0,ANG MO KIO,1.380092,103.842097,Prodigal Roasters,1.382212,103.841347,Coffee Shop
1,ANG MO KIO,1.380092,103.842097,Ho-Ree Roasted Food 好味烧腊,1.381160,103.840977,Chinese Restaurant
2,ANG MO KIO,1.380092,103.842097,Vegetarian @ 630 AMK Kopitiam,1.380721,103.841327,Vegetarian / Vegan Restaurant
3,ANG MO KIO,1.380092,103.842097,Ang Mo Kio Town Garden West,1.376039,103.844528,Park
4,ANG MO KIO,1.380092,103.842097,Shanghai Ren Jia 上海人家,1.377068,103.841329,Shanghai Restaurant
5,ANG MO KIO,1.380092,103.842097,Mr Teh Tarik Eating House,1.378458,103.836685,Food Court
6,ANG MO KIO,1.380092,103.842097,Sumo Big Prawn,1.381121,103.840778,Noodle House
7,ANG MO KIO,1.380092,103.842097,Charcos The Flaming Chicken,1.378220,103.836871,Australian Restaurant
8,ANG MO KIO,1.380092,103.842097,Nanyang Polytechnic Auditorium,1.380186,103.848298,College Auditorium
9,ANG MO KIO,1.380092,103.842097,Mr Teh Tarik,1.372168,103.845602,Halal Restaurant


In [64]:
amk.to_csv("amk.csv")

## SEMBAWANG

In [65]:
# @hidden_cell
#Defining Foursquare credentials and version
CLIENT_ID='5GYCGVHIUVEFMT10VNQK1EHYW4XAJIILTGVWUATISPOTWQ3K'
CLIENT_SECRET='UXNQUS20PDIZSMSXW1NMHZXC55XGYYG5IRUCYXHSAD4TXVSR'
VERSION='20180605'

In [66]:
#Create GET request url

latitude=semb.loc[0,'area_latitude']
longitude=semb.loc[0,'area_longitude']

LIMIT=200
radius=1000

url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID,
CLIENT_SECRET,
VERSION,
latitude,
longitude,
radius,
LIMIT)

url


'https://api.foursquare.com/v2/venues/explore?&client_id=5GYCGVHIUVEFMT10VNQK1EHYW4XAJIILTGVWUATISPOTWQ3K&client_secret=UXNQUS20PDIZSMSXW1NMHZXC55XGYYG5IRUCYXHSAD4TXVSR&v=20180605&ll=1.4557470456642672,103.818827683365&radius=1000&limit=200'

In [67]:
#send GET request url and examine results
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '613da917655c20456e56b3b3'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Sembawang',
  'headerFullLocation': 'Sembawang, Singapore',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 31,
  'suggestedBounds': {'ne': {'lat': 1.4647470546642762,
    'lng': 103.82781378863032},
   'sw': {'lat': 1.446747036664258, 'lng': 103.80984157809968}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '567d4285498ef1df28cfb57b',
       'name': 'Starbucks',
       'location': {'address': '#01-22, Sun Plaza',
        'crossStreet': '30 Sembawang Drive',
        'lat': 1.4481281132935184,
        'lng': 103.81965198217407,
        'labeledLatLngs':

In [68]:
#function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']
    

In [69]:
#Cleaning the json and structuring it into a Pandas DataFrame
venues = results['response']['groups'][0]['items']

#flatten json
nearby_venues = pd.json_normalize(venues)
                       
#filter columns
filtered_columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues=nearby_venues.loc[:,filtered_columns]

#filter category for each row
nearby_venues['venue.categories']=nearby_venues.apply(get_category_type, axis=1)
                           
#clean columns
nearby_venues.columns=[col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(200)

,name,categories,lat,lng
0,Starbucks,Coffee Shop,1.448128,103.819652
1,McDonald's,Fast Food Restaurant,1.448552,103.819927
2,White Restaurant,Chinese Restaurant,1.448468,103.819869
3,Happy Hawkers Coffee Shop,Food Court,1.452429,103.816921
4,Swensen's,Bistro,1.448177,103.819471
5,NTUC FairPrice,Supermarket,1.448071,103.819419
6,Bus Stop 58121 (Blk 424A Car Park),Bus Station,1.453338,103.819547
7,Ya Kun Kaya Toast 亞坤,Coffee Shop,1.448128,103.819441
8,Chang Cheng Mee Wah,Food Court,1.452247,103.811192
9,Sushi Express,Japanese Restaurant,1.448118,103.819348


In [70]:
#create 3 additional columns to indicate respective planning area and its longitude and latitude
semb=nearby_venues.assign(planning_area='SEMBAWANG',
                           area_latitude=latitude,
                           area_longitude=longitude)


In [71]:
#Repositioning the columns and renaming the column names

semb=semb[['planning_area','area_latitude','area_longitude','name','lat','lng','categories']]
semb.rename(columns={'name':'venue_name',
            'lat':'venue_latitude',
             'lng':'venue_longitude',
             'categories':'venue_categories'}, inplace=True)

In [72]:
#check data
semb

,planning_area,area_latitude,area_longitude,venue_name,venue_latitude,venue_longitude,venue_categories
0,SEMBAWANG,1.455747,103.818828,Starbucks,1.448128,103.819652,Coffee Shop
1,SEMBAWANG,1.455747,103.818828,McDonald's,1.448552,103.819927,Fast Food Restaurant
2,SEMBAWANG,1.455747,103.818828,White Restaurant,1.448468,103.819869,Chinese Restaurant
3,SEMBAWANG,1.455747,103.818828,Happy Hawkers Coffee Shop,1.452429,103.816921,Food Court
4,SEMBAWANG,1.455747,103.818828,Swensen's,1.448177,103.819471,Bistro
5,SEMBAWANG,1.455747,103.818828,NTUC FairPrice,1.448071,103.819419,Supermarket
6,SEMBAWANG,1.455747,103.818828,Bus Stop 58121 (Blk 424A Car Park),1.453338,103.819547,Bus Station
7,SEMBAWANG,1.455747,103.818828,Ya Kun Kaya Toast 亞坤,1.448128,103.819441,Coffee Shop
8,SEMBAWANG,1.455747,103.818828,Chang Cheng Mee Wah,1.452247,103.811192,Food Court
9,SEMBAWANG,1.455747,103.818828,Sushi Express,1.448118,103.819348,Japanese Restaurant


In [73]:
semb.to_csv("semb.csv")

## QUEENSTOWN

In [74]:
# @hidden_cell
#Defining Foursquare credentials and version
CLIENT_ID='5GYCGVHIUVEFMT10VNQK1EHYW4XAJIILTGVWUATISPOTWQ3K'
CLIENT_SECRET='UXNQUS20PDIZSMSXW1NMHZXC55XGYYG5IRUCYXHSAD4TXVSR'
VERSION='20180605'

In [75]:
# Create GET request url
latitude=queenstown.loc[0,'area_latitude']
longitude=queenstown.loc[0,'area_longitude']

LIMIT=200
radius=1000

url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID,
CLIENT_SECRET,
VERSION,
latitude,
longitude,
radius,
LIMIT)

url


'https://api.foursquare.com/v2/venues/explore?&client_id=5GYCGVHIUVEFMT10VNQK1EHYW4XAJIILTGVWUATISPOTWQ3K&client_secret=UXNQUS20PDIZSMSXW1NMHZXC55XGYYG5IRUCYXHSAD4TXVSR&v=20180605&ll=1.289072870703827,103.78455889386018&radius=1000&limit=200'

In [76]:
#send GET request url and examine results
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '613da9260265b55f8822c3b2'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Central Region',
  'headerFullLocation': 'Central Region, Singapore',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 55,
  'suggestedBounds': {'ne': {'lat': 1.2980728797038361,
    'lng': 103.79354437287707},
   'sw': {'lat': 1.280072861703818, 'lng': 103.7755734148433}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '598c60e5fd16bb0cfe4dbe3f',
       'name': 'The Good Beer Company',
       'location': {'address': '#01-23 Ascent',
        'crossStreet': 'Savourworld',
        'lat': 1.2917369090991357,
        'lng': 103.78426632723797,
        'labele

In [77]:
#function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    
    if len(categories_list)==0:
        return None
    else:
        
        return categories_list[0]['name']
    

In [78]:
#Cleaning the json and structuring it into a Pandas DataFrame
venues = results['response']['groups'][0]['items']

#flatten json
nearby_venues = pd.json_normalize(venues)
                       
#filter columns
filtered_columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues=nearby_venues.loc[:,filtered_columns]

#filter category for each row
nearby_venues['venue.categories']=nearby_venues.apply(get_category_type, axis=1)
                           
#clean columns
nearby_venues.columns=[col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(200)

,name,categories,lat,lng
0,The Good Beer Company,Beer Bar,1.291737,103.784266
1,Starbucks,Coffee Shop,1.289674,103.789908
2,Starbucks,Coffee Shop,1.287492,103.782755
3,NUH Medical Centre (One@KentRidge),Medical Center,1.293742,103.784358
4,Diamond Kitchen 鑽石小廚,Chinese Restaurant,1.287539,103.790444
5,Kent Ridge Park,Park,1.283898,103.790803
6,Mr. Bean,Snack Place,1.294492,103.784332
7,Breadstick's Cafe,Vietnamese Restaurant,1.288648,103.783249
8,Khansama Tandoori Village,Indian Restaurant,1.287894,103.790232
9,Subway,Sandwich Place,1.294449,103.784377


In [79]:
#create 3 additional columns to indicate respective planning area and its longitude and latitude
queenstown=nearby_venues.assign(planning_area='QUEENSTOWN',
                           area_latitude=latitude,
                           area_longitude=longitude)


In [80]:
#Repositioning the columns and renaming the column names
queenstown=queenstown[['planning_area','area_latitude','area_longitude','name','lat','lng','categories']]
queenstown.rename(columns={'name':'venue_name',
            'lat':'venue_latitude',
             'lng':'venue_longitude',
             'categories':'venue_categories'}, inplace=True)

In [81]:
#check data
queenstown

,planning_area,area_latitude,area_longitude,venue_name,venue_latitude,venue_longitude,venue_categories
0,QUEENSTOWN,1.289073,103.784559,The Good Beer Company,1.291737,103.784266,Beer Bar
1,QUEENSTOWN,1.289073,103.784559,Starbucks,1.289674,103.789908,Coffee Shop
2,QUEENSTOWN,1.289073,103.784559,Starbucks,1.287492,103.782755,Coffee Shop
3,QUEENSTOWN,1.289073,103.784559,NUH Medical Centre (One@KentRidge),1.293742,103.784358,Medical Center
4,QUEENSTOWN,1.289073,103.784559,Diamond Kitchen 鑽石小廚,1.287539,103.790444,Chinese Restaurant
5,QUEENSTOWN,1.289073,103.784559,Kent Ridge Park,1.283898,103.790803,Park
6,QUEENSTOWN,1.289073,103.784559,Mr. Bean,1.294492,103.784332,Snack Place
7,QUEENSTOWN,1.289073,103.784559,Breadstick's Cafe,1.288648,103.783249,Vietnamese Restaurant
8,QUEENSTOWN,1.289073,103.784559,Khansama Tandoori Village,1.287894,103.790232,Indian Restaurant
9,QUEENSTOWN,1.289073,103.784559,Subway,1.294449,103.784377,Sandwich Place


In [82]:
queenstown.to_csv("queenstown.csv")

In [83]:
#Create a list of dataframes and concatenate them
ev_list = [bedok,jw,cck,punggol,amk,semb,queenstown ]  
all_ev = pd.concat(ev_list)

In [84]:
#check data
all_ev

,planning_area,area_latitude,area_longitude,venue_name,venue_latitude,venue_longitude,venue_categories
0,BEDOK,1.324047,103.928409,Bedok Chwee Kueh 勿洛水粿,1.324903,103.930250,Chinese Restaurant
1,BEDOK,1.324047,103.928409,Haidilao Hot Pot 海底捞火锅,1.324299,103.929104,Hotpot Restaurant
2,BEDOK,1.324047,103.928409,Ya Kun Kaya Toast 亞坤,1.324095,103.929198,Coffee Shop
3,BEDOK,1.324047,103.928409,Song Zhou Luo Bo Gao 松洲箩卜糕,1.324836,103.930520,Breakfast Spot
4,BEDOK,1.324047,103.928409,FairPrice Fínest,1.324140,103.929260,Supermarket
...,...,...,...,...,...,...,...
50,QUEENSTOWN,1.289073,103.784559,Normanton Park Swimming Pool Seats,1.287149,103.792575,Pool
51,QUEENSTOWN,1.289073,103.784559,Bayville Condominium,1.281375,103.787678,Residential Building (Apartment / Condo)
52,QUEENSTOWN,1.289073,103.784559,Look Out Point | Kent Ridge Park,1.283374,103.790919,Scenic Lookout
53,QUEENSTOWN,1.289073,103.784559,Yong Kee Famous Fishball Noodles,1.288819,103.793211,Noodle House


In [85]:
#save to csv 
all_ev.to_csv('all_ev.csv', index=False)

In [86]:
#Check number of rows and columns
all_ev.shape

(413, 7)

#### We will be leaving out Tengah as it is currently an undeveloped area and the query resulted in a warning that there were insufficient results.

#### THE END